In [70]:
# Import the necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [71]:
# first set the working directory. This code will be changed based on the relative location of the data files 
# on the local drive of the computer executing the command. 
os.chdir('C:\\Users\\belincoln\\Documents\\! CBP\\!User Fees\\!! Goal 1 Dashboards')

# Works well for Jupyter Notebooks, can be configured in Spyder using file explorer. 
collections = pd.read_excel(os.path.join('Source Emails & Source Files','Files','Collections',
                                         'COBRA_Air','Collections cc495 - FY13 - FY18.xls'))

In [72]:
# Begin Data Cleaning
# delete columns and rows that contain only na
collections = collections.dropna(axis=0, how = 'all')
collections = collections.dropna(axis=1, how = 'all')

In [73]:
# Delete the first two rows and only keep the Period Column and Collection Columns
collections = collections.iloc[2:,[1,-1,-3,-4]]

In [74]:
# Make first row column headers
collections.columns = collections.iloc[0,:]

In [75]:
# Make sure all columns have a name! Rename first column 'Period'
# Delete first two rows
collections.columns.values[0] = 'Period'

collections = collections.iloc[2:,:]

In [76]:
# Delete rows that contain sums for each company (don't want to double count collections)
# Also delete additional NaNs
collections = collections.dropna(axis = 0, subset = ['Period'])

In [77]:
# Sum Interest, Penalty, and Principal Collection Amounts
collections['Collections'] = collections[['Applied Penalty Amount', 'Applied Interest Amount',
       'Applied Principal Amount']].sum(axis = 1)

In [78]:
# remove tilda from index (unclear why it exists in the first place)
collections['Period'] = collections['Period'].str.rstrip('~')
# remove extra space from index
collections['Period'] = collections['Period'].str.rstrip(' ')

In [79]:
# Groups Collections based on Collection Period, across all companies
collections = collections.groupby(collections['Period']).sum()

In [80]:
# Remove audit payments
collections= collections[~collections.index.str.contains("\*")]

# Add an additional column that shows remittance period (independent of year)
collections['Remittance Period'] = collections.index.str.split('20').str[0]


# Create Calendar Year Column
collections['Calendar Year'] = collections.index.str.split(')').str[1]
# Turn Years into integers
collections['Calendar Year'] = collections['Calendar Year'].astype(int)

# Filter on years not a part of analysis
years = [2012,2013,2014,2015,2016,2017,2018]
collections = collections[collections['Calendar Year'].isin(years)]

In [81]:
#remove collection data for which we don't have workload data
searchfor = ['Qtr 01 \(Jan-Mar\) 2012','Qtr 02 \(Apr-Jun\) 2012', 'Qtr 03 \(Jul-Sept\) 2012','Qtr 04 \(Oct-Dec\) 2018']
collections = collections[~collections.index.str.contains('|'.join(searchfor))]


In [82]:
#%% Read Workload Data
workload = pd.read_excel(os.path.join('Source Emails & Source Files','Files','Workload',
                                      'COBRA_Air','FY09-fy18-passenger data air and cruise.xlsx'))

In [83]:
# Drop Cruise and Total Rows
workload.drop([7,8],axis = 0,inplace=True)

In [84]:
workload

,Data Id,Data Elements - National,10/1/2008,11/1/2008,12/1/2008,1/1/2009,2/1/2009,3/1/2009,4/1/2009,5/1/2009,...,12/1/2017,1/1/2018,2/1/2018,3/1/2018,4/1/2018,5/1/2018,6/1/2018,7/1/2018,8/1/2018,9/1/2018
0,USC00011,Commercial Aircraft Pax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NATIONAL,5437977.0,4986218.0,5446923.0,5628757.0,4590526.0,5649832.0,5848890.0,5324342.0,...,7794441.0,8235566.0,6785954.0,8600528.0,8662139.0,8608627.0,9165631.0,10615039.0,10164128.0,8102772.0
2,MPP00013,Private Aircraft Passengers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NATIONAL,31531.0,32307.0,29799.0,36279.0,27772.0,33705.0,29347.0,20400.0,...,22546.0,25712.0,23464.0,30079.0,26404.0,22555.0,24388.0,26656.0,23076.0,20025.0
4,USCP0011,Preclear Air Ports Pax/Crew combined (Calc),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NATIONAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1596994.0,1565813.0,1530305.0,1884844.0,1745714.0,1680540.0,1845968.0,2073376.0,2034479.0,1651825.0
6,FPPASSEN,Foreign Psngrs (Cruise vessels),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
names = workload.iloc[[0,2,4],[0,1]]
workload.iloc[[1,3,5],[0,1]] = names

In [86]:
workload.iloc[[1,3,5],[0,1]] = names
names

,Data Id,Data Elements - National
0,USC00011,Commercial Aircraft Pax
2,MPP00013,Private Aircraft Passengers
4,USCP0011,Preclear Air Ports Pax/Crew combined (Calc)


In [87]:
workload

,Data Id,Data Elements - National,10/1/2008,11/1/2008,12/1/2008,1/1/2009,2/1/2009,3/1/2009,4/1/2009,5/1/2009,...,12/1/2017,1/1/2018,2/1/2018,3/1/2018,4/1/2018,5/1/2018,6/1/2018,7/1/2018,8/1/2018,9/1/2018
0,USC00011,Commercial Aircraft Pax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,5437977.0,4986218.0,5446923.0,5628757.0,4590526.0,5649832.0,5848890.0,5324342.0,...,7794441.0,8235566.0,6785954.0,8600528.0,8662139.0,8608627.0,9165631.0,10615039.0,10164128.0,8102772.0
2,MPP00013,Private Aircraft Passengers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,31531.0,32307.0,29799.0,36279.0,27772.0,33705.0,29347.0,20400.0,...,22546.0,25712.0,23464.0,30079.0,26404.0,22555.0,24388.0,26656.0,23076.0,20025.0
4,USCP0011,Preclear Air Ports Pax/Crew combined (Calc),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1596994.0,1565813.0,1530305.0,1884844.0,1745714.0,1680540.0,1845968.0,2073376.0,2034479.0,1651825.0
6,FPPASSEN,Foreign Psngrs (Cruise vessels),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
workload.iloc[:,[0,1]] = workload.iloc[:,[0,1]].ffill(axis = 0)

In [89]:
workload = workload.iloc[[1,3,5],:]

In [90]:
workload

,Data Id,Data Elements - National,10/1/2008,11/1/2008,12/1/2008,1/1/2009,2/1/2009,3/1/2009,4/1/2009,5/1/2009,...,12/1/2017,1/1/2018,2/1/2018,3/1/2018,4/1/2018,5/1/2018,6/1/2018,7/1/2018,8/1/2018,9/1/2018
1,USC00011,Commercial Aircraft Pax,5437977.0,4986218.0,5446923.0,5628757.0,4590526.0,5649832.0,5848890.0,5324342.0,...,7794441.0,8235566.0,6785954.0,8600528.0,8662139.0,8608627.0,9165631.0,10615039.0,10164128.0,8102772.0
3,MPP00013,Private Aircraft Passengers,31531.0,32307.0,29799.0,36279.0,27772.0,33705.0,29347.0,20400.0,...,22546.0,25712.0,23464.0,30079.0,26404.0,22555.0,24388.0,26656.0,23076.0,20025.0
5,USCP0011,Preclear Air Ports Pax/Crew combined (Calc),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1596994.0,1565813.0,1530305.0,1884844.0,1745714.0,1680540.0,1845968.0,2073376.0,2034479.0,1651825.0


In [91]:
# Turn Series into a dataframe. Rename the first column "Workload"
workload = workload.to_frame()
workload.columns = ['Workload']

AttributeError: 'DataFrame' object has no attribute 'to_frame'

In [ ]:
# Create Calendar Year and Month Columns
workload['Month'] = workload.index.str.split('/').str[0]
workload['Calendar Year'] = workload.index.str.split('/').str[2]


In [ ]:
workload.tail(30)

In [ ]:
# Filter on years not a part of analysis
years = ['2012','2013','2014','2015','2016','2017','2018']
workload = workload[workload['Calendar Year'].isin(years)]

In [ ]:
type(workload.index[0])

In [ ]:
# Build out Remittance Period Columns
conditions = [(workload['Month'] == '1'), (workload['Month'] == '2'), (workload['Month'] == '3'), 
              (workload['Month'] == '4'), (workload['Month'] == '5'), (workload['Month'] == '6'),
              (workload['Month'] == '7'),(workload['Month'] == '8'),(workload['Month'] == '9'),
              (workload['Month'] == '10'),(workload['Month'] == '11'),(workload['Month'] == '12')] 
choices = ['Qtr 01 (Jan-Mar)','Qtr 01 (Jan-Mar)','Qtr 01 (Jan-Mar)',
           'Qtr 02 (Apr-Jun)','Qtr 02 (Apr-Jun)','Qtr 02 (Apr-Jun)',
           'Qtr 03 (Jul-Sept)','Qtr 03 (Jul-Sept)', 'Qtr 03 (Jul-Sept)',
           'Qtr 04 (Oct-Dec)','Qtr 04 (Oct-Dec)','Qtr 04 (Oct-Dec)']
workload['Remittance Period'] = np.select(conditions, choices, default='error')

In [ ]:
# Reset index to Remittance Period plus Calendar Year to merge with Collection data
workload.index = workload['Remittance Period'] + ' ' + workload['Calendar Year']
# Drop unnecssary columns
workload.drop(['Calendar Year', 'Month','Remittance Period'], inplace = True, axis=1)


In [ ]:
workload.tail(30)

In [ ]:
# Sum on Remittance Period
workload = workload.groupby(workload.index).sum()

In [ ]:
workload_collections = pd.merge(workload,collections,how = 'inner', left_index = True, right_index = True)
#%%

In [ ]:
workload_collections

In [ ]:
corr = workload_collections.corr()

In [ ]:
corr